# Spark Train Logistic Regression


Train Logistic Regression classifier with Apache SparkML

In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

if [ $version == '37' ]; then
    pip install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1
elif [ $version == '38' ]; then
    pip install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1
else
    echo 'Currently only python 3.6 and 3.8 is supported, in case you need a different version please open an issue at https://github.com/elyra-ai/component-library/issues'
    exit -1
fi

In [2]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import logging
import shutil
import site
import sys
import wget
import re

In [3]:
if sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 and 3.8 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/elyra-ai/component-library/issues')

In [4]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [5]:
parameters = list(
  map(
      lambda s: re.sub('$', '"', s),
      map(
          lambda s: s.replace('=', '="'),
          filter(
              lambda s: s.find('=') > -1 and bool(re.match('[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
              sys.argv
          )
      )
  )
)

for parameter in parameters:
    logging.warning('Parameter: '+parameter) 
    exec(parameter)

In [6]:
conf = SparkConf().setMaster(master)
if sys.version[0:3] == '3.7':
    conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

21/10/12 17:43:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/12 17:43:47 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/12 17:43:47 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/10/12 17:43:47 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
df = spark.read.parquet(data_dir + data_parquet)

In [8]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [9]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [10]:
splits = df.randomSplit([0.8, 0.2], seed=1)
df_train = splits[0]
df_test = splits[1]

In [11]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [12]:
# import org.apache.spark.ml.feature.RFormula

# val irisData = spark.read.format("csv").option("header", "true").
# option("inferSchema", "true").load("Iris.csv")
# val irisSchema = irisData.schema

# val rFormula = new RFormula().setFormula("Species ~ .")
# val dtClassifier = new DecisionTreeClassifier().setLabelCol(
# rFormula.getLabelCol).setFeaturesCol(rFormula.getFeaturesCol)
# val pipeline = new Pipeline().setStages(Array(rFormula, dtClassifier))

In [13]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [14]:
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])

In [15]:
model = pipeline.fit(df_train)

21/10/12 17:44:28 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/10/12 17:44:28 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [16]:
prediction = model.transform(df_train)

In [17]:
binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")

binEval.evaluate(prediction)

0.2071181828870239

In [18]:
pmmlBuilder = PMMLBuilder(sc, df_train, model)
pmmlBuilder.buildFile(data_dir + model_target)

'/resources/labs/BD0231EN/final_project/claimed/component-library/train/../../data/model.xml'

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
lr2 = LogisticRegression()
pipeline2 = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr2])
grid = ParamGridBuilder().addGrid(lr2.maxIter, [10, 100, 1000]).addGrid(lr2.regParam, [0.01, 0.5, 2.0]).addGrid(lr2.elasticNetParam, [0.0, 0.5, 1.0]).build()
crossval = CrossValidator(estimator=pipeline2,
                          estimatorParamMaps=grid,
                          evaluator=binEval,
                          numFolds=2)

cvModel = crossval.fit(df_train)

21/10/12 20:03:07 WARN execution.CacheManager: Asked to cache already cached data.
21/10/12 20:03:07 WARN execution.CacheManager: Asked to cache already cached data.


In [ ]:
modelAcc = max(cvModel.avgMetrics)
print(modelAcc)

In [ ]:
bestM = cvModel.bestModel
for stage in bestM.stages:
    print(stage.extractParamMap())